In [1]:
using PyPlot
using Interact
using Distributions

pygui(true)
ion()
fig = figure();

In [2]:
function cloud!(x, res, start, n, d)
    dim = size(x, 2)
    for i=1:n
        res[start+i, 1] = x[1] + rand(d)
        res[start+i, 2] = x[2] + rand(d)
    end
end

function circle!(x, r, res, start, n, d)
    dim = size(x, 2)
    for i=1:n
        angle = rand() * 2 * pi
        res[start+i, 1] = x[1] + cos(angle) * r + rand(d)
        res[start+i, 2] = x[2] + sin(angle) * r + rand(d)
    end    
end

function lin_sep_two_clouds()
    x = zeros(100, 2)
    cloud!([0 0], x,  0, 50, Normal(0, 0.2))
    cloud!([1 1], x, 50, 50, Normal(0, 0.2))

    y = repeat([1; -1], inner=50)
    x, y    
end

function two_cocentric()
    x = zeros(150, 2)
    cloud!( [0 0],      x,  0,  50, Normal(0, 0.1))
    circle!([0 0], 1.5, x, 50, 100, Normal(0, 0.1))

    y = repeat([1; -1; -1], inner=50)
    x, y
end

two_cocentric (generic function with 1 method)

In [3]:
function transform_grid!(f, xgrid, ygrid, zgrid)
    N = size(xgrid,1)

    for i in 1:N
        for j in 1:N
            y = f([xgrid[i, j], ygrid[i, j], 1])
                
            xgrid[i:i,j:j] = y[1]
            ygrid[i:i,j:j] = y[2]
            zgrid[i:i,j:j] = y[3]
        end
    end
end

function transform(f, X)
    N = size(X, 1)
    
    xd = zeros(N)
    yd = zeros(N)
    zd = zeros(N)
    
    for i in 1:N
        y = f([X[i, 1], X[i, 2], 1])
        
        xd[i] = y[1]
        yd[i] = y[2]
        zd[i] = y[3]
    end
    
    xd,yd,zd
end

transform (generic function with 1 method)

In [8]:
X, y = lin_sep_two_clouds();

@manipulate for α=linspace(pi/2, pi, 10)
    withfig(fig) do
        xlim(-.5,1.5)
        ylim(-.5,1.5)
        axis("off")
        plot(.5 + 10*[cos(α), cos(α+pi)], .5 + 10*[sin(α), sin(α+pi)],color="black",linestyle="dashed",linewidth=0.7)
        
        scatter(X[:,1][y.==1], X[:,2][y.==1], marker="x", c="black")
        scatter(X[:,1][y.==-1], X[:,2][y.==-1], marker="o", edgecolor="black", facecolor=(0,0,0,0))
        
        #scatter(X[:,1], X[:,2], s=10000, edgecolor="black", facecolor=(0,0,0,0))
    end
end;

Interact.Options{:SelectionSlider,Float64}(Signal{Float64}(2.2689280275926285, nactions=1),"α",2.2689280275926285,"2.2689280275926285",Interact.OptionDict(DataStructures.OrderedDict("1.5707963267948966"=>1.5708,"1.7453292519943295"=>1.74533,"1.9198621771937623"=>1.91986,"2.0943951023931953"=>2.0944,"2.2689280275926285"=>2.26893,"2.443460952792061"=>2.44346,"2.617993877991494"=>2.61799,"2.792526803190927"=>2.79253,"2.9670597283903604"=>2.96706,"3.141592653589793"=>3.14159…),Dict(1.5708=>"1.5707963267948966",1.74533=>"1.7453292519943295",1.91986=>"1.9198621771937623",2.26893=>"2.2689280275926285",2.96706=>"2.9670597283903604",2.79253=>"2.792526803190927",2.44346=>"2.443460952792061",2.61799=>"2.617993877991494",3.14159=>"3.141592653589793",2.0944=>"2.0943951023931953"…)),Any[],Any[],true,"horizontal")

In [4]:
N = 20
xrange, yrange = linspace(-2, 2, N), linspace(-2, 2, N)

f = (x) -> [x[1]^2, x[2]^2, sqrt(2)x[1]x[2]]
X, y = two_cocentric();

@manipulate for step=slider(1:20, value=1),
                showing=[:none => 0,
                         :space_boundary => 1,
                         :both => 2,
                         :intersection => 3
                        ]
    
    λ = linspace(0, 1, 20)[step]
    g = (x) -> (1 - λ) * x + λ * f(x)

    xgrid = repmat(xrange', N, 1)
    ygrid = repmat(yrange, 1, N)
    zgrid = zeros(N,N)

    xsep = repmat(xrange', N, 1) + .5
    ysep = -repmat(xrange', N, 1) + .5
    zsep = 2*repmat(yrange, 1, N)
    
    angles = repmat(linspace(-pi, pi, 20), 20, 2)
    xb,yb,zb = transform((x) -> g([cos(x[1]), sin(x[1]), 1]), angles)
    
    transform_grid!(g, xgrid, ygrid, zgrid)

    xd, yd, zd = transform(g, X)
    
    withfig(fig, clear=false) do
        cla()
        
        plt[:gca](projection="3d")[:_axis3don] = false
        mesh(xgrid, ygrid, zgrid, color="black", linestyles="dashed", linewidths=0.3)
        
        if showing == 1
            mesh(xsep, ysep, zsep, color="red", linestyles="dashed", linewidths=0.3)
        elseif showing == 2
            mesh(xsep, ysep, zsep, color="red", linestyles="dashed", linewidths=0.3)
            mesh(xb, yb, zb, color="red", linestyles="dashed", linewidths=0.3)
        elseif showing == 3
            mesh(xb, yb, zb, color="red", linestyles="dashed", linewidths=0.3)
        end
        
        scatter3D(xd[y.==1], yd[y.==1], zd[y.==1], marker=".")
        scatter3D(xd[y.==-1], yd[y.==-1], zd[y.==-1], marker="x")
    end
end;

Interact.Slider{Int64}(Signal{Int64}(1, nactions=1),"",1,1:20,"horizontal",true,"d",true)

Interact.Options{:ToggleButtons,Int64}(Signal{Int64}(0, nactions=1),"showing",0,"none",Interact.OptionDict(DataStructures.OrderedDict("none"=>0,"space_boundary"=>1,"both"=>2,"intersection"=>3),Dict(0=>"none",2=>"both",3=>"intersection",1=>"space_boundary")),Any[],Any[],true,"horizontal")